In [63]:
import requests
import json
import re

In [86]:
# url 접속
DB_url = 'http://13.125.204.95:8080/api/workspaces/project-info'
response = requests.get(url = DB_url)
data = response.json()['data']

# 방법 1: JSON string을 .json 확장자로 저장 (기본)
json_string = json.dumps(data, ensure_ascii=False, indent=2)
with open('project_info_DB.json', 'w', encoding='utf-8') as f:
    f.write(json_string)

In [87]:
from fastapi import APIRouter, HTTPException
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re
import json
import ast

router = APIRouter()

##################################################
# DB에 이으는 과정 필요! (DB 연결 구역)
##################################################

# 임시 데이터로 작성
# project_info DB 연결필요

def read_DB() :
    with open("project_info_DB.json", "r", encoding="utf-8") as f: 
        user_project_info = json.load(f)

    solutionIdea_lst = []
    stack_lst = []

    for i in range(len(user_project_info)) :
        solutionIdea_lst.append(user_project_info[i]['problem_solving']['solutionIdea'])
        stack_lst.append(user_project_info[i]['technology_stack'])

    send_dataset = pd.DataFrame({
        'subject': solutionIdea_lst,
        'stack': stack_lst
    })

    return send_dataset
####################################################


In [90]:
with open("project_info_DB.json", "r", encoding="utf-8") as f: 
    user_project_info = ast.literal_eval(json.load(f))

projectID_lst = []
solutionIdea_lst = []
stack_lst = []

for i in range(len(user_project_info)) :
    projectID_lst.append(user_project_info[i]['projectInfoId'])
    solutionIdea_lst.append(user_project_info[i]['problemSolving']['solutionIdea'])
    stack_lst.append(user_project_info[i]['technologyStack'])

send_dataset = pd.DataFrame(index = projectID_lst, data = {
    'subject': solutionIdea_lst,
    'stack': stack_lst
})

send_dataset

,subject,stack
11,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공,"[javaScript, java]"
12,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있도록 지원,"[javaScript, java]"
15,스터디 그룹 내에서 과제 관리와 퀴즈 기능을 통해 학습 효과를 극대화,"[javaScript, java]"
17,스터디 그룹 내 학습 내용을 복습할 수 있는 기능 제공 및 소통 강화,"[스프링, 리액트, MySql]"
25,"이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고, 스터디 그룹을 통해 협력할 수...","[React, Spring Boot, MariaDB, Redis, Docker, K..."
21,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 스터디 그룹 내에서의 소통을 강화...,"[HTML, CSS, JavaScript, Java (Servlet, JSP), M..."
26,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,"[React, TypeScript, Spring Boot, PostgreSQL, A..."
24,"본 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹 운영을 지원하며, 체크리...","[HTML, CSS, JavaScript, Java (Spring Framework..."
29,스터디 마스터는 사용자 친화적인 인터페이스를 통해 학생들이 스터디 그룹을 쉽게 생성...,"[React.js, Node.js, MySQL, AWS (S3, EC2), Sock..."
30,"본 프로젝트는 컴퓨터 비전을 활용하여 리듬게임 영상을 프레임 단위로 분석하고, 노트...","[Python (OpenCV, Scikit-learn), Spring Boot, R..."


In [66]:
# 텍스트 전처리 함수
def preprocess_text(text):
    if pd.isna(text):
        return ""
    # HTML 태그 제거
    text = re.sub('<.*?>', '', str(text))
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)
    text = re.sub('[^가-힣a-zA-Z0-9\s]', ' ', text)
    # 연속된 공백을 하나로 변환
    text = re.sub('\s+', ' ', text)
    return text.strip()

# TfidfVectorizer 진행
vectorizer = TfidfVectorizer(
    max_features=1000,  # 최대 특성 수
    ngram_range=(1, 2),  # 1-gram과 2-gram 사용
    min_df=1,  # 최소 문서 빈도
    max_df=0.95,  # 최대 문서 빈도 (너무 자주 나오는 단어 제외)
    stop_words=None
)

In [67]:
class ProjectRecommender :
    def __init__(self,  sample_df, vectorizer):
        self.sample_df = sample_df
        self.vectorizer = vectorizer

    def filter_stack_recomend_subjects(self, input_text ,target_stack, top_k = 10) :
        
        # 공통되는 요소가 하나라도 있는 행
        has_stack = self.sample_df[self.sample_df['stack'].apply(lambda x: bool(set(x) & set(target_stack)))]

        # 원본 인덱스 저장
        original_indices = has_stack.index.tolist()

        # 벡터화 실행
        tfidf_matrix = vectorizer.fit_transform(has_stack['processed_subject'])

        # 사용자의 특정 개요의 유사도 기반 추천
        processed_input = preprocess_text(input_text)
        input_vector = vectorizer.transform([processed_input])
        similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
        similar_indices = similarities.argsort()[::-1][:top_k]

        # 원본 DataFrame의 인덱스로 변환
        original_similar_indices = [original_indices[i] for i in similar_indices]

        return original_similar_indices

In [68]:
project_info = json.loads(data)[10]

In [ ]:
# 데이터 준비
send_dataset = read_DB()

sample_df = send_dataset.copy()

# subject 컬럼 전처리
sample_df['processed_subject'] = sample_df['subject'].apply(preprocess_text)

# 추천 시스템 초기화
recommender = ProjectRecommender(sample_df, vectorizer)

# 요청에서 데이터 추출
input_text = project_info['problem_solving']['solutionIdea']
target_stack = project_info['technology_stack']
top_k = 10

recommend_id = recommender.filter_stack_recomend_subjects(input_text, target_stack, top_k)

similer_ID = {"project_ID" : recommend_id}